### Problem 1 (50 points) 

Vapor-liquid equilibria data are correlated using two adjustable parameters $A_{12}$ and $A_{21}$ per binary
mixture. For low pressures, the equilibrium relation can be formulated as:

$$
\begin{aligned}
p = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{water}^{sat}\\
& + x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{1,4 dioxane}^{sat}.
\end{aligned}
$$

Here the saturation pressures are given by the Antoine equation

$$
\log_{10}(p^{sat}) = a_1 - \frac{a_2}{T + a_3},
$$

where $T = 20$($^{\circ}{\rm C}$) and $a_{1,2,3}$ for a water - 1,4 dioxane
system is given below.

|             | $a_1$     | $a_2$      | $a_3$     |
|:------------|:--------|:---------|:--------|
| Water       | 8.07131 | 1730.63  | 233.426 |
| 1,4 dioxane | 7.43155 | 1554.679 | 240.337 |


The following table lists the measured data. Recall that in a binary system $x_1 + x_2 = 1$.

|$x_1$ | 0.0 | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
|:-----|:--------|:---------|:--------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
|$p$| 28.1 | 34.4 | 36.7 | 36.9 | 36.8 | 36.7 | 36.5 | 35.4 | 32.9 | 27.7 | 17.5 |

Estimate $A_{12}$ and $A_{21}$ using data from the above table: 

1. Formulate the least square problem; 
2. Since the model is nonlinear, the problem does not have an analytical solution. Therefore, solve it using the gradient descent or Newton's method implemented in HW1; 
3. Compare your optimized model with the data. Does your model fit well with the data?

---

### Problem 2 (50 points) 

Solve the following problem using Bayesian Optimization:
$$
    \min_{x_1, x_2} \quad \left(4-2.1x_1^2 + \frac{x_1^4}{3}\right)x_1^2 + x_1x_2 + \left(-4 + 4x_2^2\right)x_2^2,
$$
for $x_1 \in [-3,3]$ and $x_2 \in [-2,2]$. A tutorial on Bayesian Optimization can be found [here](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/).





In [52]:
# A simple example of using PyTorch for gradient descent

import torch as t
from torch.autograd import Variable

# Define a variable, make sure requires_grad=True so that PyTorch can take gradient with respect to this variable
x = Variable(t.tensor([1.0, 0.0]), requires_grad=True)

# Define a loss
loss = (x[0] - 1)**2 + (x[1] - 2)**2

# Take gradient
loss.backward()

# Check the gradient. numpy() turns the variable from a PyTorch tensor to a numpy array.
x.grad.numpy()

tensor(4., grad_fn=<AddBackward0>)

In [42]:
# Let's examine the gradient at a different x.
x.data = t.tensor([2.0, 1.0])
loss = (x[0] - 1)**2 + (x[1] - 2)**2
loss.backward()
x.grad.numpy()

array([ 2., -6.], dtype=float32)

In [96]:
# Here is a code for gradient descent without line search

import torch as t
from torch.autograd import Variable

x = Variable(t.tensor([1.0, 0.0]), requires_grad=True)

# Fix the step size
a = 0.01

# Start gradient descent
for i in range(1000):  # TODO: change the termination criterion
    loss = (x[0] - 1)**2 + (x[1] - 2)**2
    loss.backward()
    
    # no_grad() specifies that the operations within this context are not part of the computational graph, i.e., we don't need the gradient descent algorithm itself to be differentiable with respect to x
    with t.no_grad():
        x -= a * x.grad
        
        # need to clear the gradient at every step, or otherwise it will accumulate...
        x.grad.zero_()
        
print(x.data.numpy())
print(loss.data.numpy())

AttributeError: 'NoneType' object has no attribute 'zero_'

In [120]:
import torch as t
from torch.autograd import Variable
import math
# Defining variables we want to optimize
A12 = Variable(t.tensor([2.0]), requires_grad=True)
A21 = Variable(t.tensor([2.0]), requires_grad=True)
#A = Variable(t.tensor([1.0,0.0]), requires_grad=True)
#A12 = 5
#A21 = 6
# Setting constants
T = 20+273
a1_w = 8.07131
a2_w = 1730.63
a3_w = 233.426
a1_d = 7.43155
a2_d = 1554.679
a3_d = 240.337
p_satw = pow(10,(a1_w - (a2_w)/(T+a3_w)))
p_satd = pow(10,(a1_d - (a2_d)/(T+a3_d)))
x1 = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
#x1 = 0
p = [28.1,34.4,36.7,36.9,36.8,36.7,36.5,35.4,32.9,27.7,17.5]
#p = 28.1
pd = lambda x1,x2,A12,A21: x1*t.exp(A12*((A21*x2)/(A12*x1+A21*x2))**2)*p_satw + x2*t.exp(A21*((A12*x1)/(A12*x1 + A21*x2))**2)*p_satd
#x2 = 1.0
for i in range(len(x1)):
    if i == 0:
        x2 = [1-x1[i]]
    else:
        x2.append(1-x1[i])
for i in range(len(x1)):
    if i == 0:
        loss = (p[i] - x1[i]*t.exp(A12*(A21*x2[i]/(A12*1+A21*x2[i]))**2)*p_satw + x2[i]*t.exp(A21*(A12*x2[i]/(A12*x1[i] + A21*x2[i]))**2)*p_satd)**2
        loss.backward()
        A12c = [print(A12.grad.numpy())]
        A21c = [print(A21.grad.numpy())]
    else: 
        loss = (p[i] - x1[i]*t.exp(A12*(A21*x2[i]/(A12*1+A21*x2[i]))**2)*p_satw + x2[i]*t.exp(A21*(A12*x2[i]/(A12*x1[i] + A21*x2[i]))**2)*p_satd)**2
        loss.backward()
        A12c.append(print(A12.grad.numpy()))
        A21c.append(print(A21.grad.numpy())
        



SyntaxError: unexpected EOF while parsing (<ipython-input-120-986f6e314fef>, line 43)

In [121]:
loss

tensor([3.6928e+09], grad_fn=<PowBackward0>)

In [85]:
pd(x1,x2,3.1906513e+10,-1.5953256e+10)

OverflowError: math range error